# Image Segmentation Task 1
#### Welcome to the first task of Image Segmentation. Image segmentation is the process of partitioning the image into a set of pixels representing an object. In this task, you will be introduced to the problem of image segmentation and programming pipeline involved in image segmentation.

For the purpose of this task we will be using PASCAL VOC datset. The dataset contains a total of 2913 images with segmentation annotations. Code in the cell below will download the code and extract the dataset.

### 1.1 Loading the dataset

In [ ]:
import os
from os.path import join as pjoin
import collections
import json
import torch
#import imageio
import numpy as np
import scipy.misc as m
import scipy.io as io
import matplotlib.pyplot as plt
import glob

from PIL import Image
from tqdm import tqdm
from torch.utils import data
from torchvision import transforms


class pascalVOCDataset(data.Dataset):
    """Data loader for the Pascal VOC semantic segmentation dataset.

    Annotations from both the original VOC data (which consist of RGB images
    in which colours map to specific classes) and the SBD (Berkely) dataset
    (where annotations are stored as .mat files) are converted into a common
    `label_mask` format.  Under this format, each mask is an (M,N) array of
    integer values from 0 to 21, where 0 represents the background class.

    The label masks are stored in a new folder, called `pre_encoded`, which
    is added as a subdirectory of the `SegmentationClass` folder in the
    original Pascal VOC data layout.

    A total of five data splits are provided for working with the VOC data:
        train: The original VOC 2012 training data - 1464 images
        val: The original VOC 2012 validation data - 1449 images
        trainval: The combination of `train` and `val` - 2913 images
        train_aug: The unique images present in both the train split and
                   training images from SBD: - 8829 images (the unique members
                   of the result of combining lists of length 1464 and 8498)
        train_aug_val: The original VOC 2012 validation data minus the images
                   present in `train_aug` (This is done with the same logic as
                   the validation set used in FCN PAMI paper, but with VOC 2012
                   rather than VOC 2011) - 904 images
    """

    def __init__(
        self,
        root,
        sbd_path=None,
        split="train_aug",
        is_transform=False,
        img_size=512,
        augmentations=None,
        img_norm=True,
        test_mode=False,
    ):
        self.root = root
        self.sbd_path = sbd_path
        self.split = split
        self.is_transform = is_transform
        self.augmentations = augmentations
        self.img_norm = img_norm
        self.test_mode = test_mode
        self.n_classes = 21
        self.mean = np.array([104.00699, 116.66877, 122.67892])
        self.files = collections.defaultdict(list)
        self.img_size = img_size if isinstance(img_size, tuple) else (img_size, img_size)

        if not self.test_mode:
            for split in ["train", "val", "trainval"]:
                path = pjoin(self.root, "ImageSets/Segmentation", split + ".txt")
                file_list = tuple(open(path, "r"))
                file_list = [id_.rstrip() for id_ in file_list]
                self.files[split] = file_list
            self.setup_annotations()

        self.tf = transforms.Compose(
            [
                # add more trasnformations as you see fit
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ]
        )

    def __len__(self):
        return len(self.files[self.split])

    def __getitem__(self, index):
        im_name = self.files[self.split][index]
        im_path = pjoin(self.root, "JPEGImages", im_name + ".jpg")
        lbl_path = pjoin(self.root, "SegmentationClass/pre_encoded", im_name + ".png")
        im = Image.open(im_path)
        lbl = Image.open(lbl_path)
        if self.augmentations is not None:
            im, lbl = self.augmentations(im, lbl)
        if self.is_transform:
            im, lbl = self.transform(im, lbl)
        return im, torch.clamp(lbl, max=20)

    def transform(self, img, lbl):
        if self.img_size == ("same", "same"):
            pass
        else:
            img = img.resize((self.img_size[0], self.img_size[1]))  # uint8 with RGB mode
            lbl = lbl.resize((self.img_size[0], self.img_size[1]))
        img = self.tf(img)
        lbl = torch.from_numpy(np.array(lbl)).long()
        lbl[lbl == 255] = 0
        return img, lbl

    def get_pascal_labels(self):
        """Load the mapping that associates pascal classes with label colors

        Returns:
            np.ndarray with dimensions (21, 3)
        """
        return np.asarray(
            [
                [0, 0, 0],
                [128, 0, 0],
                [0, 128, 0],
                [128, 128, 0],
                [0, 0, 128],
                [128, 0, 128],
                [0, 128, 128],
                [128, 128, 128],
                [64, 0, 0],
                [192, 0, 0],
                [64, 128, 0],
                [192, 128, 0],
                [64, 0, 128],
                [192, 0, 128],
                [64, 128, 128],
                [192, 128, 128],
                [0, 64, 0],
                [128, 64, 0],
                [0, 192, 0],
                [128, 192, 0],
                [0, 64, 128],
            ]
        )

    def encode_segmap(self, mask):
        """Encode segmentation label images as pascal classes

        Args:
            mask (np.ndarray): raw segmentation label image of dimension
              (M, N, 3), in which the Pascal classes are encoded as colours.

        Returns:
            (np.ndarray): class map with dimensions (M,N), where the value at
            a given location is the integer denoting the class index.
        """
        mask = mask.astype(int)
        label_mask = np.zeros((mask.shape[0], mask.shape[1]), dtype=np.int16)
        for ii, label in enumerate(self.get_pascal_labels()):
            label_mask[np.where(np.all(mask == label, axis=-1))[:2]] = ii
        label_mask = label_mask.astype(int)
        # print(np.unique(label_mask))
        return label_mask

    def decode_segmap(self, label_mask, plot=False):
        """Decode segmentation class labels into a color image

        Args:
            label_mask (np.ndarray): an (M,N) array of integer values denoting
              the class label at each spatial location.
            plot (bool, optional): whether to show the resulting color image
              in a figure.

        Returns:
            (np.ndarray, optional): the resulting decoded color image.
        """
        label_colours = self.get_pascal_labels()
        r = label_mask.copy()
        g = label_mask.copy()
        b = label_mask.copy()
        for ll in range(0, self.n_classes):
            r[label_mask == ll] = label_colours[ll, 0]
            g[label_mask == ll] = label_colours[ll, 1]
            b[label_mask == ll] = label_colours[ll, 2]
        rgb = np.zeros((label_mask.shape[0], label_mask.shape[1], 3))
        rgb[:, :, 0] = r / 255.0
        rgb[:, :, 1] = g / 255.0
        rgb[:, :, 2] = b / 255.0
        if plot:
            plt.imshow(rgb)
            plt.show()
        else:
            return rgb

    def setup_annotations(self):
        """Sets up Berkley annotations by adding image indices to the
        `train_aug` split and pre-encode all segmentation labels into the
        common label_mask format (if this has not already been done). This
        function also defines the `train_aug` and `train_aug_val` data splits
        according to the description in the class docstring
        """
        sbd_path = self.sbd_path
        target_path = pjoin(self.root, "SegmentationClass/pre_encoded")
        if not os.path.exists(target_path):
            os.makedirs(target_path)
        train_aug = self.files["train"]

        # keep unique elements (stable)
        train_aug = [train_aug[i] for i in sorted(np.unique(train_aug, return_index=True)[1])]
        self.files["train_aug"] = train_aug
        set_diff = set(self.files["val"]) - set(train_aug)  # remove overlap
        self.files["train_aug_val"] = list(set_diff)

        pre_encoded = glob.glob(pjoin(target_path, "*.png"))
        expected = np.unique(self.files["train_aug"] + self.files["val"]).size

        if len(pre_encoded) != expected:
            print("Pre-encoding segmentation masks...")

            for ii in tqdm(self.files["trainval"]):
                fname = ii + ".png"
                lbl_path = pjoin(self.root, "SegmentationClass", fname)
                lbl = self.encode_segmap(m.imread(lbl_path))
                lbl = m.toimage(lbl, high=lbl.max(), low=lbl.min())
                m.imsave(pjoin(target_path, fname), lbl)

        assert expected == 2913, "unexpected dataset sizes"

In [ ]:
N_CLASSES = 21

### 1.2 Define the model architecture(2.0 point)
In this section you have the freedom to decide your own model. Keep in mind though, to perform image segmentation, you need to implement an architecture that does pixel level classification i.e. for each pixel in the image you need to predict the probability of it belonging to one of the 21 categories.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# Wrappers Around nn.Conv2d and nn.ConvTranspose2d
# They add mandatory batch norm and relu activation:
# s.t. they compute f(x) = ReLU(Conv(BatchNorm(x))
# Also add way of inspecting output sizes given input size
class Conv2d(nn.Conv2d):
    
    def __init__(self, in_dims, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.in_dims = in_dims
        self.bn = nn.BatchNorm2d(self.in_channels)
    
    def forward(self, x):
        return F.relu(super().forward(self.bn(x)))
    
    def _out_dim(self, i=0):
        return int(
            (self.in_dims[i] + self.padding[i] - self.dilation[i] * (self.kernel_size[i] - 1) - 1)
            / self.stride[i] + 1
        )
    
    def out_dims(self):
        return (self._out_dim(0), self._out_dim(1))
        
class DeConv2d(nn.ConvTranspose2d):
    
    def __init__(self, in_dims, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.in_dims = in_dims
        self.bn = nn.BatchNorm2d(self.in_channels)
    
    def forward(self, x):
        return F.relu(super().forward(self.bn(x)))
    
    def _out_dim(self, i=0):
        return ((self.in_dims[i] - 1) * self.stride[i] 
                - 2 * self.padding[i]
                + self.dilation[i] * (self.kernel_size[i] - 1)
                + self.output_padding[i] + 1)
    
    def out_dims(self):
        return (self._out_dim(0), self._out_dim(1))        
        

In [ ]:
DeConv2d((125, 125), 1, 1, kernel_size=3, stride=2, dilation=3).out_dims()

In [ ]:
class Conv2dBlock(nn.Module):
    
    def __init__(self, in_dims, in_channels, out_channels, depth,
                 *args, max_pool=False, **kwargs):
        super().__init__()
        
        self.in_dims = in_dims
        self.max_pool = max_pool
        
        modules = []
        
        # initial convolution
        modules.append(Conv2d(in_dims, in_channels, out_channels, *args, **kwargs))
        
        # all the other following conv layers
        for i in range(1, depth):
            _in_dims = modules[i-1].out_dims()
            modules.append(Conv2d(_in_dims, out_channels, out_channels, *args, **kwargs))
        
        # optional max pooling
        if max_pool:
            modules.append(nn.MaxPool2d(2))
            
        self.net = nn.Sequential(*modules)
        
        #assert all(d % 2 == 0 for d in self.out_dims()), print(self.out_dims())
        
    def forward(self, x):
        return self.net(x)
    
    def out_dims(self):
        if self.max_pool:
            return tuple(d // 2 for d in self.net[-2].out_dims())
        else:
            return self.net[-1].out_dims()
            
    
class Encoder(nn.Module):
    
    def __init__(self, in_dims, in_channels, out_channels, conv_depth, path_depth):
        
        super().__init__()       
        modules = []
        
        for k in range(path_depth):
            
            modules.append(Conv2dBlock(in_dims, in_channels, out_channels, conv_depth,
                                       kernel_size=3, stride=1, padding=1,max_pool=k != (path_depth - 1)))
            in_channels = out_channels
            out_channels = out_channels * 2          
            in_dims = modules[-1].out_dims()
            
        self.net = nn.Sequential(*modules)
        
    def out_dims(self):
        return self.net[-1].out_dims()
    
    def forward(self, x):
        return self.net(x)
    
class Decoder(nn.Module):

    def __init__(self, in_dims, in_channels, conv_depth, path_depth):
        
        super().__init__()
        modules = []
        
        for k in range(path_depth-1):
            
            modules.append(DeConv2d(in_dims, in_channels, in_channels // 2, kernel_size=2, stride=2))
            in_channels = in_channels // 2
            in_dims = modules[-1].out_dims()
            
            modules.append(Conv2dBlock(in_dims, in_channels, in_channels, conv_depth,
                                       kernel_size=3, stride=1, padding=1))
            in_dims = modules[-1].out_dims()
            
        self.net = nn.Sequential(*modules)
        
    def forward(self, x):
        return self.net(x)
    
    def out_dims(self):
        return self.net[-1].out_dims()

In [ ]:
class Segnet(nn.Module):
  
    def __init__(self, in_dims=(512, 512), d0=32, conv_depth=2, path_depth=2):
        super(Segnet, self).__init__()
        
        self.encoder = Encoder(in_dims, 3, d0, conv_depth, path_depth)
        
        self.decoder = Decoder(self.encoder.out_dims(), d0*(2**(path_depth-1)), conv_depth, path_depth)
        
        self.classifier = nn.Conv2d(d0, N_CLASSES, kernel_size=1, stride=1, padding=0)

    def forward(self, x):
        x1 = self.encoder(x)
        print(x1.shape)
        x2 = self.decoder(x1)
        print(x2.shape)
        pred = self.classifier(x2)
        print(pred.shape)
        return pred
    

In [ ]:
def param_count(module):
    return sum(p.numel() for p in module.parameters())

def print_params(model):
    for name, module in model.named_modules():
        mdepth = name.count('.')
        if mdepth > 1:
            continue
        print('\t'*mdepth + name + ':' + str(param_count(module)))

param_count(nn.Conv2d(8, 8, 3))

In [ ]:
# Creating an instance of the model defined above. 
# You can modify it incase you need to pass paratemers to the constructor.
model = Segnet()
print_params(model)
model

### 1.3 Hyperparameters(0.5 points)
Define all the hyperparameters(not restricted to the three given below) that you find useful here.

In [ ]:
local_path = 'VOCdevkit/VOC2012/' # modify it according to your device
bs = 32
epochs = 1
lr=5e-4

### 1.4 Dataset and Dataloader(0.5 points)
Create the dataset using pascalVOCDataset class defined above. Use local_path defined in the cell above as root. 

In [ ]:
# dataset variable
dst = pascalVOCDataset(local_path, is_transform=True)

# dataloader variable
trainloader = torch.utils.data.DataLoader(dst, batch_size=bs, shuffle=True)

### 1.5 Loss fuction and Optimizer(1.0 point)
Define below with the loss function you think would be most suitable for segmentation task. You are free to choose any optimizer to train the network.

In [ ]:
# loss function
loss_f = nn.CrossEntropyLoss(weight=weights)

# optimizer variable
opt = torch.optim.Adam(model.parameters(), lr=lr)

### 1.6 Training the model(3.0 points)
Your task here is to complete the code below to perform a training loop and save the model weights after each epoch of training.

In [ ]:
import logging
logging.basicConfig(format='%(asctime)s %(message)s', level=logging.INFO)

In [ ]:
for i_epoch in range(1, epochs + 1):
    logging.info("Epoch %d / %d", i_epoch, epochs)
    for i, d in enumerate(trainloader):
        x, target = d
        pred = model(x)
        loss = loss_f(pred, target)
        
        opt.zero_grad()
        loss.backward()
        opt.step()
        
        logging.info("Batch %d / %d - Loss: %.3f", i, len(trainloader), loss.item())

### 1.7 Evaluate your model(1.5 points)
In this section you have to implement the evaluation metrics for your model. Calculate the values of F1-score, dice coefficient and AUC-ROC score on the data you used for training. You can use external packages like scikit-learn to compute above metrics.

In [ ]:
def evaluate(ground_truth, predictions):
    
    return f1_score, auc_score, dice_coeeficient

### 1.8 Plot the evaluation metrics against epochs(1.0)
In section 1.6 we saved the weights of the model after each epoch. In this section, you have to calculate the evaluation metrics after each epoch of training by loading the weights for each epoch. Once you have calculated the evaluation metrics for each epoch, plot them against the epochs.

### 1.9 Visualize results(0.5 points)
For any 10 images in the dataset, show the images along the with their segmentation mask.

In [ ]:
import random
i = random.randint(0,100)

x, y = dst[i]
x = x.unsqueeze(0)

pred = model(x).squeeze(0)
pred = torch.argmax(pred, dim=0)

dst.decode_segmap(pred.numpy(), plot=True)
dst.decode_segmap(y.numpy(), plot=True)

In [ ]:
total_counts = torch.zeros(N_CLASSES)
i = 0
while True:
    try:
        _, labels = dst[i]
    except:
        break
    classes, counts = torch.unique(labels, return_counts=True)
    total_counts[classes] += counts
    i = i + 1

In [ ]:
beta = 0.5
soft_counts = total_counts ** beta
weights = soft_counts.mean() / soft_counts

In [ ]:
plt.bar(np.arange(N_CLASSES), weights)

In [ ]:
weights.size()